# Dask Bags


Dask Bag implements operations like `map`, `filter`, `groupby` and aggregations on collections of Python objects. It does this in parallel and in small memory using Python iterators. It is similar to a parallel version of itertools or a Pythonic version of the PySpark RDD.

Dask Bags are often used to do simple preprocessing on log files, JSON records, or other user defined Python objects.

Full API documentation is available here: http://docs.dask.org/en/latest/bag-api.html

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client, progress
client = Client(n_workers=4, threads_per_worker=1)
client

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46423 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:34729 Dashboard: http://127.0.0.1:46423/status,Cluster Workers: 4 Cores: 4 Memory: 7.29 GB


## Create Random Data

We create a random set of record data and store it to disk as many JSON files.  This will serve as our data for this notebook.

In [2]:
import dask
import json
import os

os.makedirs('data', exist_ok=True)              # Create data/ directory

b = dask.datasets.make_people()                 # Make records of people
b.map(json.dumps).to_textfiles('data/*.json')   # Encode as JSON, write to disk

['/home/runner/work/dask-examples/dask-examples/data/0.json',
 '/home/runner/work/dask-examples/dask-examples/data/1.json',
 '/home/runner/work/dask-examples/dask-examples/data/2.json',
 '/home/runner/work/dask-examples/dask-examples/data/3.json',
 '/home/runner/work/dask-examples/dask-examples/data/4.json',
 '/home/runner/work/dask-examples/dask-examples/data/5.json',
 '/home/runner/work/dask-examples/dask-examples/data/6.json',
 '/home/runner/work/dask-examples/dask-examples/data/7.json',
 '/home/runner/work/dask-examples/dask-examples/data/8.json',
 '/home/runner/work/dask-examples/dask-examples/data/9.json']

## Read JSON data

Now that we have some JSON data in a file lets take a look at it with Dask Bag and Python JSON module.

In [3]:
!head -n 2 data/0.json

{"age": 59, "name": ["Olevia", "Frazier"], "occupation": "Shop Manager", "telephone": "277.930.0371", "address": {"address": "1383 Hillway Promenade", "city": "Gretna"}, "credit-card": {"number": "3464 767413 59033", "expiration-date": "02/21"}}
{"age": 30, "name": ["Emely", "Gomez"], "occupation": "Hosiery Worker", "telephone": "1-358-738-0556", "address": {"address": "902 Pretor Run", "city": "Flower Mound"}, "credit-card": {"number": "2601 1391 2520 6685", "expiration-date": "09/17"}}


In [4]:
import dask.bag as db
import json

b = db.read_text('data/*.json').map(json.loads)
b

dask.bag<loads, npartitions=10>

In [5]:
b.take(2)

({'age': 59,
  'name': ['Olevia', 'Frazier'],
  'occupation': 'Shop Manager',
  'telephone': '277.930.0371',
  'address': {'address': '1383 Hillway Promenade', 'city': 'Gretna'},
  'credit-card': {'number': '3464 767413 59033', 'expiration-date': '02/21'}},
 {'age': 30,
  'name': ['Emely', 'Gomez'],
  'occupation': 'Hosiery Worker',
  'telephone': '1-358-738-0556',
  'address': {'address': '902 Pretor Run', 'city': 'Flower Mound'},
  'credit-card': {'number': '2601 1391 2520 6685',
   'expiration-date': '09/17'}})

## Map, Filter, Aggregate

We can process this data by filtering out only certain records of interest, mapping functions over it to process our data, and aggregating those results to a total value.

In [6]:
b.filter(lambda record: record['age'] > 30).take(2)  # Select only people over 30

({'age': 59,
  'name': ['Olevia', 'Frazier'],
  'occupation': 'Shop Manager',
  'telephone': '277.930.0371',
  'address': {'address': '1383 Hillway Promenade', 'city': 'Gretna'},
  'credit-card': {'number': '3464 767413 59033', 'expiration-date': '02/21'}},
 {'age': 44,
  'name': ['Bula', 'Newman'],
  'occupation': 'Hairdresser',
  'telephone': '1-788-321-8035',
  'address': {'address': '146 Galewood Plantation', 'city': 'Ponca City'},
  'credit-card': {'number': '4805 9520 4978 4714',
   'expiration-date': '01/18'}})

In [7]:
b.map(lambda record: record['occupation']).take(2)  # Select the occupation field

('Shop Manager', 'Hosiery Worker')

In [8]:
b.count().compute()  # Count total number of records

10000

## Chain computations

It is common to do many of these steps in one pipeline, only calling `compute` or `take` at the end.

In [9]:
result = (b.filter(lambda record: record['age'] > 30)
           .map(lambda record: record['occupation'])
           .frequencies(sort=True)
           .topk(10, key=1))
result

dask.bag<topk-aggregate, npartitions=1>

As with all lazy Dask collections, we need to call `compute` to actually evaluate our result.  The `take` method used in earlier examples is also like `compute` and will also trigger computation.

In [10]:
result.compute()

[('Taxi Controller', 16),
 ('Cable TV Installer', 15),
 ('Production Engineer', 15),
 ('Anaesthetist', 14),
 ('Warehousewoman', 13),
 ('Lift Engineer', 13),
 ('Scientific Officer', 13),
 ('Radiologist', 13),
 ('Scrap Dealer', 13),
 ('Publisher', 13)]

## Transform and Store

Sometimes we want to compute aggregations as above, but sometimes we want to store results to disk for future analyses.  For that we can use methods like `to_textfiles` and `json.dumps`, or we can convert to Dask Dataframes and use their storage systems, which we'll see more of in the next section.

In [11]:
(b.filter(lambda record: record['age'] > 30)  # Select records of interest
  .map(json.dumps)                            # Convert Python objects to text
  .to_textfiles('data/processed.*.json'))     # Write to local disk

['/home/runner/work/dask-examples/dask-examples/data/processed.0.json',
 '/home/runner/work/dask-examples/dask-examples/data/processed.1.json',
 '/home/runner/work/dask-examples/dask-examples/data/processed.2.json',
 '/home/runner/work/dask-examples/dask-examples/data/processed.3.json',
 '/home/runner/work/dask-examples/dask-examples/data/processed.4.json',
 '/home/runner/work/dask-examples/dask-examples/data/processed.5.json',
 '/home/runner/work/dask-examples/dask-examples/data/processed.6.json',
 '/home/runner/work/dask-examples/dask-examples/data/processed.7.json',
 '/home/runner/work/dask-examples/dask-examples/data/processed.8.json',
 '/home/runner/work/dask-examples/dask-examples/data/processed.9.json']

## Convert to Dask Dataframes

Dask Bags are good for reading in initial data, doing a bit of pre-processing, and then handing off to some other more efficient form like Dask Dataframes.  Dask Dataframes use Pandas internally, and so can be much faster on numeric data and also have more complex algorithms.  

However, Dask Dataframes also expect data that is organized as flat columns.  It does not support nested JSON data very well (Bag is better for this).

Here we make a function to flatten down our nested data structure, map that across our records, and then convert that to a Dask Dataframe.

In [12]:
b.take(1)

({'age': 59,
  'name': ['Olevia', 'Frazier'],
  'occupation': 'Shop Manager',
  'telephone': '277.930.0371',
  'address': {'address': '1383 Hillway Promenade', 'city': 'Gretna'},
  'credit-card': {'number': '3464 767413 59033', 'expiration-date': '02/21'}},)

In [13]:
def flatten(record):
    return {
        'age': record['age'],
        'occupation': record['occupation'],
        'telephone': record['telephone'],
        'credit-card-number': record['credit-card']['number'],
        'credit-card-expiration': record['credit-card']['expiration-date'],
        'name': ' '.join(record['name']),
        'street-address': record['address']['address'],
        'city': record['address']['city']   
    }

b.map(flatten).take(1)

({'age': 59,
  'occupation': 'Shop Manager',
  'telephone': '277.930.0371',
  'credit-card-number': '3464 767413 59033',
  'credit-card-expiration': '02/21',
  'name': 'Olevia Frazier',
  'street-address': '1383 Hillway Promenade',
  'city': 'Gretna'},)

In [14]:
df = b.map(flatten).to_dataframe()
df.head()

,age,occupation,telephone,credit-card-number,credit-card-expiration,name,street-address,city
0,59,Shop Manager,277.930.0371,3464 767413 59033,02/21,Olevia Frazier,1383 Hillway Promenade,Gretna
1,30,Hosiery Worker,1-358-738-0556,2601 1391 2520 6685,09/17,Emely Gomez,902 Pretor Run,Flower Mound
2,44,Hairdresser,1-788-321-8035,4805 9520 4978 4714,01/18,Bula Newman,146 Galewood Plantation,Ponca City
3,41,Sales Executive,(695) 842-6942,4437 9053 4721 5852,07/20,Lewis Padilla,56 Forest Knolls Close,Ithaca
4,17,Outdoor Pursuits,901-540-5781,4698 6716 8087 3467,09/19,Max Lyons,163 Battery Caulfield Field,Bentonville


We can now perform the same computation as before, but now using Pandas and Dask dataframe.

In [15]:
df[df.age > 30].occupation.value_counts().nlargest(10).compute()

Taxi Controller        16
Cable TV Installer     15
Production Engineer    15
Anaesthetist           14
Lift Engineer          13
Publisher              13
Scientific Officer     13
Scrap Dealer           13
Radiologist            13
Warehousewoman         13
Name: occupation, dtype: int64

## Learn More

You may be interested in the following links:

-  [Dask Bag Documentation](https://docs.dask.org/en/latest/bag.html)
-  [API Documentation](http://docs.dask.org/en/latest/bag-api.html)
-  [dask tutorial](https://github.com/dask/dask-tutorial), notebook 02, for a more in-depth introduction.